Video Link: https://drive.google.com/file/d/11PKWgWapBinZ9S6lQG0BRtibTmxq7avv/view?usp=sharing

In [ ]:
!git clone https://github.com/GavinKerrigan/conf_matrix_and_calibration.git

Cloning into 'conf_matrix_and_calibration'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 44 (delta 12), reused 22 (delta 3), pack-reused 0
Unpacking objects: 100% (44/44), done.


In [ ]:
!pip install attrdict deepdish pyro-ppl uncertainty-calibration

     |████████████████████████████████| 718 kB 6.7 MB/s 
  Created wheel for uncertainty-calibration: filename=uncertainty_calibration-0.0.9-py3-none-any.whl size=12286 sha256=d6be5457d6e161e62720a6313ebbe56995c67b746550637b3dfc60d2074f6a25
  Stored in directory: /root/.cache/pip/wheels/d2/b4/25/3eee8271e3c0d7b2d1f3e4e3312daf1c981405a8526860eb09
Successfully built uncertainty-calibration


In [ ]:
!cp -r ./conf_matrix_and_calibration/* .
!rm -r conf_matrix_and_calibration

In [ ]:
!cp ./experiments/calibration_experiment.py .
!cp ./experiments/calibrate_combo_experiment.py .
!rm -r experiments

In [ ]:
!mkdir data
!cp -r ./cifar10h data/
!rm -r cifar10h
!rm -r sample_data

### 2.1
**Q1** Reproducing results from paper




In [ ]:
!mkdir output
!mkdir output/cifar10h
!mkdir output/cifar10h/final
!mkdir output/cifar10h/final/fully_sup_CI
!mkdir output/cifar10h/final/calibrate_comb_MAP

In [ ]:
import torch
import numpy as np

In [ ]:
from calibration_experiment import run_experiment_cifar10
from metrics import get_NLL, get_ECE, get_cw_ECE

In [ ]:
seed = 9658
torch.manual_seed(seed)
np.random.seed(seed)

calibration_methods = ['none', 'confusion', 'temperature scaling', 'ensemble temperature scaling', 'imax binning']
"""
calibration_metrics = {'ECE width': lambda probs, y: get_ECE(probs, y, mode='width'),
                        'ECE mass': lambda probs, y: get_ECE(probs, y, mode='mass'),
                        'cwECE thresh width': lambda probs, y: get_cw_ECE(probs, y, mode='width'),
                        'cwECE thresh mass': lambda probs, y: get_cw_ECE(probs, y, mode='mass'),
                        'cwECE nothresh width': lambda probs, y: get_cw_ECE(probs, y, mode='width',
                                                                            threshold_mode=None),
                        'cwECE nothresh mass': lambda probs, y: get_cw_ECE(probs, y, mode='mass',
                                                                          threshold_mode=None),
                        'kumar MCE': get_MCE,
                        'kumar MCE (bin)': lambda probs, y: cal.get_binning_ce(probs, y,
                                                                              p=1, debias=False, mode='marginal'),
                        'kumar MCE (scale)': lambda probs, y: cal.lower_bound_scaling_ce(probs, y,
                                                                                        p=1, debias=False,
                                                                                        mode='marginal'),
                        'kumar ECE': cal.get_ece}
"""
calibration_metrics = {'ECE (W)': lambda probs, y: get_ECE(probs, y, mode='width'),
                        'ECE (M)': lambda probs, y: get_ECE(probs, y, mode='mass'),
                        'cwECE (WT)': lambda probs, y: get_cw_ECE(probs, y, mode='width'),
                        'cwECE (MT)': lambda probs, y: get_cw_ECE(probs, y, mode='mass'),
                        'cwECE (WNT)': lambda probs, y: get_cw_ECE(probs, y, mode='width', threshold_mode=None),
                        'cwECE (MNT)': lambda probs, y: get_cw_ECE(probs, y, mode='mass', threshold_mode=None),
                        'NLL': get_NLL}

args = {'n_runs': 25,
        'test_size': 0.3,
        'calibration_methods': calibration_methods,
        'calibration_metrics': calibration_metrics,
        'seed': seed
        }

In [ ]:
out_fpath = './output/cifar10h/final/fully_sup_CI/'
run_experiment_cifar10(out_fpath=out_fpath, experiment_args=args, seed=seed)

Models:   0%|          | 0/4 [00:00<?, ?it/s]

Runs:   0%|          | 0/25 [00:00<?, ?it/s]

Runs:   0%|          | 0/25 [00:00<?, ?it/s]

Runs:   0%|          | 0/25 [00:00<?, ?it/s]

Runs:   0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

In [ ]:
dense_net_acc = pd.read_csv('output/cifar10h/final/fully_sup_CI/densenet-bc-L190-k40_accuracy.csv').mean().apply(lambda x: (1-x)*100)
preres_net_acc = pd.read_csv('output/cifar10h/final/fully_sup_CI/preresnet-110_accuracy.csv').mean().apply(lambda x: (1-x)*100)
r_low_acc = pd.read_csv('output/cifar10h/final/fully_sup_CI/r_low_acc_accuracy.csv').mean().apply(lambda x: (1-x)*100)
res_net_acc = pd.read_csv('output/cifar10h/final/fully_sup_CI/resnet-110_accuracy.csv').mean().apply(lambda x: (1-x)*100)

In [ ]:
data1 = pd.DataFrame(dict(dense_net_acc), index=[0])
data1['model_name'] = 'DenseNet'
data2 = pd.DataFrame(dict(preres_net_acc), index=[0])
data2['model_name'] = 'PreresNet'
data3 = pd.DataFrame(dict(r_low_acc), index=[0])
data3['model_name'] = 'RLowAcc'
data4 = pd.DataFrame(dict(res_net_acc), index=[0])
data4['model_name'] = 'ResNetAcc'
data = pd.concat([data1, data2, data3, data4])
data = data[['model_name', 'human', 'model', 'comb none', 'comb confusion',
       'comb temperature scaling', 'comb ensemble temperature scaling',
       'comb imax binning']]

#### ***Appendix C*** : Calibration Methods and Uncalibrated Combinations Results

In [ ]:
data

,model_name,human,model,comb none,comb confusion,comb temperature scaling,comb ensemble temperature scaling,comb imax binning
0,DenseNet,4.617333,3.252000,2.217333,3.388000,2.032000,2.165333,2.044000
0,PreresNet,4.617333,4.997333,2.900000,4.361333,2.452000,2.456000,2.428000
0,RLowAcc,4.617333,11.282667,4.396000,4.697333,3.741333,3.760000,3.797333
0,ResNetAcc,4.617333,6.100000,3.045333,4.712000,2.846667,2.821333,2.849333


In [ ]:
seed = 9658
torch.manual_seed(seed)
np.random.seed(seed)

calibration_methods = ['none', 'confusion', 'temperature scaling']
"""
calibration_metrics = {'ECE width': lambda probs, y: get_ECE(probs, y, mode='width'),
                        'ECE mass': lambda probs, y: get_ECE(probs, y, mode='mass'),
                        'cwECE thresh width': lambda probs, y: get_cw_ECE(probs, y, mode='width'),
                        'cwECE thresh mass': lambda probs, y: get_cw_ECE(probs, y, mode='mass'),
                        'cwECE nothresh width': lambda probs, y: get_cw_ECE(probs, y, mode='width',
                                                                            threshold_mode=None),
                        'cwECE nothresh mass': lambda probs, y: get_cw_ECE(probs, y, mode='mass',
                                                                          threshold_mode=None),
                        'kumar MCE': get_MCE,
                        'kumar MCE (bin)': lambda probs, y: cal.get_binning_ce(probs, y,
                                                                              p=1, debias=False, mode='marginal'),
                        'kumar MCE (scale)': lambda probs, y: cal.lower_bound_scaling_ce(probs, y,
                                                                                        p=1, debias=False,
                                                                                        mode='marginal'),
                        'kumar ECE': cal.get_ece}
"""
calibration_metrics = {'ECE (W)': lambda probs, y: get_ECE(probs, y, mode='width'),
                        'ECE (M)': lambda probs, y: get_ECE(probs, y, mode='mass'),
                        'cwECE (WT)': lambda probs, y: get_cw_ECE(probs, y, mode='width'),
                        'cwECE (MT)': lambda probs, y: get_cw_ECE(probs, y, mode='mass'),
                        'cwECE (WNT)': lambda probs, y: get_cw_ECE(probs, y, mode='width', threshold_mode=None),
                        'cwECE (MNT)': lambda probs, y: get_cw_ECE(probs, y, mode='mass', threshold_mode=None),
                        'NLL': get_NLL}

args = {'n_runs': 25,
        'test_size': 0.3,
        'calibration_methods': calibration_methods,
        'calibration_metrics': calibration_metrics,
        'seed': seed
        }

out_fpath = './output/cifar10h/final/calibrate_comb_MAP/'
run_experiment_cifar10(out_fpath=out_fpath, experiment_args=args, seed=seed)


Models:   0%|          | 0/4 [00:00<?, ?it/s]

Runs:   0%|          | 0/25 [00:00<?, ?it/s]

Runs:   0%|          | 0/25 [00:00<?, ?it/s]

Runs:   0%|          | 0/25 [00:00<?, ?it/s]

Runs:   0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
# dense_net_acc = pd.read_csv('output/cifar10h/final/calibrate_comb_MAP/densenet-bc-L190-k40_accuracy.csv') #.mean().apply(lambda x: (1-x)*100)
dense_net_cal = pd.read_csv('output/cifar10h/final/calibrate_comb_MAP/densenet-bc-L190-k40_calibration.csv')
# preres_net_acc = pd.read_csv('output/cifar10h/final/calibrate_comb_MAP/preresnet-110_accuracy.csv')
preres_net_cal = pd.read_csv('output/cifar10h/final/calibrate_comb_MAP/preresnet-110_calibration.csv')
# r_low_acc_acc = pd.read_csv('output/cifar10h/final/calibrate_comb_MAP/r_low_acc_accuracy.csv')
r_low_acc_cal = pd.read_csv('output/cifar10h/final/calibrate_comb_MAP/r_low_acc_calibration.csv')
# resnet_acc = pd.read_csv('output/cifar10h/final/calibrate_comb_MAP/resnet-110_accuracy.csv')
resnet_cal = pd.read_csv('output/cifar10h/final/calibrate_comb_MAP/resnet-110_calibration.csv')

In [ ]:
dense_net_cal['metric'] = dense_net_cal['metric'].replace(['ECE (M)', 'ECE (W)'],'ECE')
dense_net_cal['metric'] = dense_net_cal['metric'].replace(['cwECE (WT)', 'cwECE (MT)', 'cwECE (WNT)', 'cwECE (MNT)'],'cwECE')

In [ ]:
preres_net_cal['metric'] = preres_net_cal['metric'].replace(['ECE (M)', 'ECE (W)'],'ECE')
preres_net_cal['metric'] = preres_net_cal['metric'].replace(['cwECE (WT)', 'cwECE (MT)', 'cwECE (WNT)', 'cwECE (MNT)'],'cwECE')

In [ ]:
r_low_acc_cal['metric'] = r_low_acc_cal['metric'].replace(['ECE (M)', 'ECE (W)'],'ECE')
r_low_acc_cal['metric'] = r_low_acc_cal['metric'].replace(['cwECE (WT)', 'cwECE (MT)', 'cwECE (WNT)', 'cwECE (MNT)'],'cwECE')

In [ ]:
resnet_cal['metric'] = resnet_cal['metric'].replace(['ECE (M)', 'ECE (W)'],'ECE')
resnet_cal['metric'] = resnet_cal['metric'].replace(['cwECE (WT)', 'cwECE (MT)', 'cwECE (WNT)', 'cwECE (MNT)'],'cwECE')

In [ ]:
# dense_net_cal

In [ ]:
dense_net_data = dense_net_cal.groupby('metric').mean()
preres_net_data = preres_net_cal.groupby('metric').mean()
r_low_acc_data = r_low_acc_cal.groupby('metric').mean()
resnet_data = resnet_cal.groupby('metric').mean()

In [ ]:
dense_net_data['name'] = 'DenseNet'
preres_net_data['name'] = 'PreResNet'
r_low_acc_data['name'] = 'RLowAcc'
resnet_data['name'] = 'ResNet'

#### ***Appendix D*** : Calibration Properties of Combinations Results

In [ ]:
pd.concat([dense_net_data, preres_net_data, r_low_acc_data, resnet_data]).groupby([ 'metric', 'name']).mean()

model      comb
metric name                         
ECE    DenseNet   0.012696  0.012534
       PreResNet  0.015421  0.015144
       RLowAcc    0.025970  0.018717
       ResNet     0.017149  0.015084
NLL    DenseNet   0.230529  0.431731
       PreResNet  0.308067  0.430007
       RLowAcc    0.499046  0.489319
       ResNet     0.328382  0.434329
cwECE  DenseNet   0.011691  0.010559
       PreResNet  0.013924  0.012125
       RLowAcc    0.020884  0.014978
       ResNet     0.015469  0.012418

**OBSERVATIONS**
1. Results: 

*   The results reproduced by the author's code and the results deoicted in the paper match in the case of appendix - C.
*   But the results given in appendix-D doesn't match. Since the results shown in table are in 10^(-2), hence can be considered as minor changes. 

2. challenges:
Lot of challenges were faced to execute author's code.

*   Data path mentioned in code were wrong. All the data paths were modified in order to execute the code
*   Code was not well documented. Hence to find the related code consumed lot of time
*   Output folders weren't created. Hence got error while executing the code.

3. hyperparameters:
*   No hyperparameters were manipulated.



### 2.2
**Q2** Model multiple humans

In [ ]:
import pandas as pd
from data_utils import load_CIFAR10H
from sklearn.model_selection import train_test_split
from metrics import get_acc
import os
from combination_methods import OracleCombiner, DoubleConfusionCombiner
from calibration_experiment import _run_experiment

In [ ]:
!mkdir output-multiple-humans

In [ ]:
import numpy as np
import random

def simulate_multiple_human(human_counts, seed=0):
    rng = np.random.default_rng(seed)

    human_labels_per_input = np.sum(human_counts, axis=1)
    min_human_labels = int(min(human_labels_per_input))
    n_rows = human_counts.shape[0]
    n_classes = human_counts.shape[1]

    human_labels = np.empty(shape=(n_rows, min_human_labels))
    for row in range(n_rows):
        temp = []
        for i in range(n_classes):
            temp += [i] * int(human_counts[row, i])
        rng.shuffle(temp)
        human_labels[row, :] = temp[:min_human_labels]

    k = random.randrange(1, min_human_labels)
    return human_labels[:, k].astype(int)

In [ ]:
calibration_methods = ['none', 'confusion', 'temperature scaling', 'ensemble temperature scaling', 'imax binning']
calibration_metrics = {'ECE (W)': lambda probs, y: get_ECE(probs, y, mode='width'),
                          'ECE (M)': lambda probs, y: get_ECE(probs, y, mode='mass'),
                          'cwECE (WT)': lambda probs, y: get_cw_ECE(probs, y, mode='width'),
                          'cwECE (MT)': lambda probs, y: get_cw_ECE(probs, y, mode='mass'),
                          'cwECE (WNT)': lambda probs, y: get_cw_ECE(probs, y, mode='width', threshold_mode=None),
                          'cwECE (MNT)': lambda probs, y: get_cw_ECE(probs, y, mode='mass', threshold_mode=None),
                          'NLL': get_NLL}

In [ ]:
def voting(human):
  mode = []
  for i in range(human.shape[0]):
    row = human[i, :]
    if row[0] == row[1]:
      mode.append(row[0])
    elif row[1] == row[2]:
      mode.append(row[1])
    elif row[0] == row[2]:
      mode.append(row[0])
    else:
      choice = random.choice(row)
      mode.append(choice)
  return np.array(mode)

In [ ]:
from tqdm import tqdm

def run_experiment_cifar10(out_fpath=None, experiment_args=None, seed=0):
  model_names = ['r_low_acc', 'resnet-110', 'preresnet-110', 'densenet-bc-L190-k40']
  for model_name in tqdm(model_names, desc='Models', leave=True):
      # Specify output files
      output_file_acc = out_fpath + f'{model_name}_accuracy.csv'
      output_file_calibration = out_fpath + f'{model_name}_calibration.csv'
      assert not os.path.exists(output_file_acc), 'Output filepath already exists'
      assert not os.path.exists(output_file_calibration), 'Output filepath already exists'
      experiment_args['output_file_acc'] = output_file_acc
      experiment_args['output_file_calibration'] = output_file_calibration

      # Load data
      human_counts, model_probs, y_true = load_CIFAR10H(model_name)

      # simulating 3 humans
      y_h_1 = simulate_multiple_human(human_counts, seed)
      y_h_2 = simulate_multiple_human(human_counts, seed)
      y_h_3 = simulate_multiple_human(human_counts, seed)
      humans = np.array([y_h_1, y_h_2, y_h_3]).T
      y_h = voting(humans)
      _run_experiment(y_h=y_h, model_probs=model_probs, y_true=y_true, **experiment_args)


In [ ]:
seed = 0
args = {'n_runs': 25,
            'test_size': 0.3,
            'calibration_methods': calibration_methods,
            'calibration_metrics': calibration_metrics,
            'seed': 0
            }
out_fpath = 'output-multiple-humans/'
run_experiment_cifar10(out_fpath=out_fpath, experiment_args=args, seed=seed)

Models:   0%|          | 0/4 [00:00<?, ?it/s]

Runs:   0%|          | 0/25 [00:00<?, ?it/s]

Models:  25%|██▌       | 1/4 [01:58<05:54, 118.17s/it]

Runs:   0%|          | 0/25 [00:00<?, ?it/s]

Models:  50%|█████     | 2/4 [03:57<03:57, 118.71s/it]

Runs:   0%|          | 0/25 [00:00<?, ?it/s]

Models:  75%|███████▌  | 3/4 [05:54<01:58, 118.20s/it]

Runs:   0%|          | 0/25 [00:00<?, ?it/s]

Models: 100%|██████████| 4/4 [07:52<00:00, 118.15s/it]


In [ ]:
m_dense_net_acc = pd.read_csv('output-multiple-humans/densenet-bc-L190-k40_accuracy.csv').mean().apply(lambda x: (1-x)*100)
m_preres_net_acc = pd.read_csv('output-multiple-humans/preresnet-110_accuracy.csv').mean().apply(lambda x: (1-x)*100)
m_r_low_acc = pd.read_csv('output-multiple-humans/r_low_acc_accuracy.csv').mean().apply(lambda x: (1-x)*100)
m_res_net_acc = pd.read_csv('output-multiple-humans/resnet-110_accuracy.csv').mean().apply(lambda x: (1-x)*100)

m_data1 = pd.DataFrame(dict(m_dense_net_acc), index=[0])
m_data1['model_name'] = 'DenseNet'
m_data2 = pd.DataFrame(dict(m_preres_net_acc), index=[0])
m_data2['model_name'] = 'PreresNet'
m_data3 = pd.DataFrame(dict(m_r_low_acc), index=[0])
m_data3['model_name'] = 'RLowAcc'
m_data4 = pd.DataFrame(dict(m_res_net_acc), index=[0])
m_data4['model_name'] = 'ResNetAcc'
m_data = pd.concat([m_data1, m_data2, m_data3, m_data4])
m_data = m_data[['model_name', 'human', 'model', 'comb none', 'comb confusion',
       'comb temperature scaling', 'comb ensemble temperature scaling',
       'comb imax binning']]

m_data

,model_name,human,model,comb none,comb confusion,comb temperature scaling,comb ensemble temperature scaling,comb imax binning
0,DenseNet,2.100000,3.633333,2.233333,2.400000,1.700000,1.733333,1.766667
0,PreresNet,2.233333,5.133333,2.366667,2.533333,1.766667,1.766667,1.733333
0,RLowAcc,2.033333,11.033333,3.133333,2.233333,1.966667,2.033333,2.100000
0,ResNetAcc,1.933333,6.466667,2.366667,2.366667,1.566667,1.633333,1.633333


In [ ]:
# m_dense_net_cal = pd.read_csv('output-multiple-humans/densenet-bc-L190-k40_calibration.csv')
# m_preres_net_cal = pd.read_csv('output-multiple-humans/preresnet-110_calibration.csv')
# m_r_low_acc_cal = pd.read_csv('output-multiple-humans/r_low_acc_calibration.csv')
# m_resnet_cal = pd.read_csv('output-multiple-humans/resnet-110_calibration.csv')

# m_dense_net_cal['metric'] = m_dense_net_cal['metric'].replace(['ECE (M)', 'ECE (W)'],'ECE')
# m_dense_net_cal['metric'] = m_dense_net_cal['metric'].replace(['cwECE (WT)', 'cwECE (MT)', 'cwECE (WNT)', 'cwECE (MNT)'],'cwECE')

# m_preres_net_cal['metric'] = m_preres_net_cal['metric'].replace(['ECE (M)', 'ECE (W)'],'ECE')
# m_preres_net_cal['metric'] = m_preres_net_cal['metric'].replace(['cwECE (WT)', 'cwECE (MT)', 'cwECE (WNT)', 'cwECE (MNT)'],'cwECE')

# m_r_low_acc_cal['metric'] = m_r_low_acc_cal['metric'].replace(['ECE (M)', 'ECE (W)'],'ECE')
# m_r_low_acc_cal['metric'] = m_r_low_acc_cal['metric'].replace(['cwECE (WT)', 'cwECE (MT)', 'cwECE (WNT)', 'cwECE (MNT)'],'cwECE')

# m_resnet_cal['metric'] = m_resnet_cal['metric'].replace(['ECE (M)', 'ECE (W)'],'ECE')
# m_resnet_cal['metric'] = m_resnet_cal['metric'].replace(['cwECE (WT)', 'cwECE (MT)', 'cwECE (WNT)', 'cwECE (MNT)'],'cwECE')

# m_dense_net_data = m_dense_net_cal.groupby('metric').mean()
# m_preres_net_data = m_preres_net_cal.groupby('metric').mean()
# m_r_low_acc_data = m_r_low_acc_cal.groupby('metric').mean()
# m_resnet_data = m_resnet_cal.groupby('metric').mean()

# m_dense_net_data['name'] = 'DenseNet'
# m_preres_net_data['name'] = 'PreResNet'
# m_r_low_acc_data['name'] = 'RLowAcc'
# m_resnet_data['name'] = 'ResNet'

# pd.concat([m_dense_net_data, m_preres_net_data, m_r_low_acc_data, m_resnet_data]).groupby([ 'metric', 'name']).mean()

**OBSERVATIONS**

In the above model 3 human decision maker label for a given input. The label which is given by most of the humans are considered as the final output. And if all three provide different label, than any one label is choosen at random.

By using this model, it can be clearly shown in the above table than the accuracy of human model is incresed by 2.5%. 

More than 1 human approches can find lot of use cases. For example in medical field, it's hard to find a single expert in all fields and sometimes it can be very helpful to get results from multiple experts instead of relying on a single decision maker. And i medical field, high accuracy is most the important. 

**Q3** Creating a neural network model incorporating human labels and model predictions as input and predicting final probabilities.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical


In [ ]:
from metrics import get_NLL, get_ECE, get_cw_ECE
from sklearn.model_selection import train_test_split
import csv
from tensorflow.keras import layers


In [ ]:
def nn_model(X, y):

  model = Sequential(
      [
          layers.Dense(32, input_dim=X.shape[1], activation='relu'),
          layers.Dense(16, activation='relu'),
          layers.Dense(10, activation='softmax'),
      ]
  )
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(X, y, epochs=10, batch_size=512)
  return model

In [ ]:
calibration_methods = ['none', 'confusion', 'temperature scaling', 'ensemble temperature scaling', 'imax binning']
calibration_metrics = {'ECE (W)': lambda probs, y: get_ECE(probs, y, mode='width'),
                          'ECE (M)': lambda probs, y: get_ECE(probs, y, mode='mass'),
                          'cwECE (WT)': lambda probs, y: get_cw_ECE(probs, y, mode='width'),
                          'cwECE (MT)': lambda probs, y: get_cw_ECE(probs, y, mode='mass'),
                          'cwECE (WNT)': lambda probs, y: get_cw_ECE(probs, y, mode='width', threshold_mode=None),
                          'cwECE (MNT)': lambda probs, y: get_cw_ECE(probs, y, mode='mass', threshold_mode=None),
                          'NLL': get_NLL}

In [ ]:
from tqdm import tqdm
from utils import simulate_single_human
import os
from data_utils import load_CIFAR10H
from calibration_experiment import _run_experiment

In [ ]:
!mkdir output-team

In [ ]:
out_fpath = 'output-team/'
experiment_args = {'n_runs': 25,
            'test_size': 0.3,
            'calibration_methods': calibration_methods,
            'calibration_metrics': calibration_metrics,
            'seed': 0
            }
model_names = ['r_low_acc', 'resnet-110', 'preresnet-110', 'densenet-bc-L190-k40']
for model_name in tqdm(model_names, desc='Models', leave=True):
    # Specify output files
    output_file_acc = out_fpath + f'{model_name}_accuracy.csv'
    output_file_calibration = out_fpath + f'{model_name}_calibration.csv'
    assert not os.path.exists(output_file_acc), 'Output filepath already exists'
    assert not os.path.exists(output_file_calibration), 'Output filepath already exists'
    experiment_args['output_file_acc'] = output_file_acc
    experiment_args['output_file_calibration'] = output_file_calibration

    # Load data
    human_counts, model_probs, y_true = load_CIFAR10H(model_name)

    # Creating NN model for each model-human pair 
    X = np.concatenate([model_probs, human_counts], axis=1)

    y = to_categorical(y_true)
    y_h = simulate_single_human(human_counts, seed=0)
    model = nn_model(X, y)
    probs = model.predict(X)

    _run_experiment(y_h=y_h, model_probs=probs, y_true=y_true, **experiment_args)

Models:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1/10
20/20 [==============================] - 3s 4ms/step - loss: 3.7128 - accuracy: 0.3131
Epoch 2/10
20/20 [==============================] - 0s 3ms/step - loss: 0.9781 - accuracy: 0.6425
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.3570 - accuracy: 0.8318
Epoch 4/10
20/20 [==============================] - 0s 3ms/step - loss: 0.1836 - accuracy: 0.9866
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.1170 - accuracy: 0.9889
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0844 - accuracy: 0.9897
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0691 - accuracy: 0.9903
Epoch 8/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0598 - accuracy: 0.9909
Epoch 9/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0524 - accuracy: 0.9915
Epoch 10/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0463 - accuracy: 0.9919


Runs:   0%|          | 0/25 [00:00<?, ?it/s]

Models:  25%|██▌       | 1/4 [02:05<06:15, 125.14s/it]

Epoch 1/10
20/20 [==============================] - 1s 4ms/step - loss: 4.8808 - accuracy: 0.2171
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 1.3640 - accuracy: 0.6907
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4869 - accuracy: 0.8549
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.2522 - accuracy: 0.8899
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.1800 - accuracy: 0.9522
Epoch 6/10
20/20 [==============================] - 0s 3ms/step - loss: 0.1364 - accuracy: 0.9912
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0960 - accuracy: 0.9913
Epoch 8/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0666 - accuracy: 0.9914
Epoch 9/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0516 - accuracy: 0.9918
Epoch 10/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0423 - accuracy: 0.9920


...

Runs:   0%|          | 0/25 [00:00<?, ?it/s]

Models:  75%|███████▌  | 3/4 [05:47<01:54, 114.44s/it]

Epoch 1/10
20/20 [==============================] - 1s 4ms/step - loss: 4.7546 - accuracy: 0.1454
Epoch 2/10
20/20 [==============================] - 0s 3ms/step - loss: 1.4962 - accuracy: 0.5342
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5825 - accuracy: 0.8782
Epoch 4/10
20/20 [==============================] - 0s 3ms/step - loss: 0.2846 - accuracy: 0.9872
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.1622 - accuracy: 0.9898
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.1000 - accuracy: 0.9908
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0702 - accuracy: 0.9911
Epoch 8/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0567 - accuracy: 0.9918
Epoch 9/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0487 - accuracy: 0.9918
Epoch 10/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0434 - accuracy: 0.9921


Runs:   0%|          | 0/25 [00:00<?, ?it/s]

Models: 100%|██████████| 4/4 [07:39<00:00, 114.88s/it]


In [ ]:
# X.shape

In [ ]:
m_dense_net_acc = pd.read_csv('output-team/densenet-bc-L190-k40_accuracy.csv').mean().apply(lambda x: (1-x)*100)
m_preres_net_acc = pd.read_csv('output-team/preresnet-110_accuracy.csv').mean().apply(lambda x: (1-x)*100)
m_r_low_acc = pd.read_csv('output-team/r_low_acc_accuracy.csv').mean().apply(lambda x: (1-x)*100)
m_res_net_acc = pd.read_csv('output-team/resnet-110_accuracy.csv').mean().apply(lambda x: (1-x)*100)

m_data1 = pd.DataFrame(dict(m_dense_net_acc), index=[0])
m_data1['model_name'] = 'DenseNet'
m_data2 = pd.DataFrame(dict(m_preres_net_acc), index=[0])
m_data2['model_name'] = 'PreresNet'
m_data3 = pd.DataFrame(dict(m_r_low_acc), index=[0])
m_data3['model_name'] = 'RLowAcc'
m_data4 = pd.DataFrame(dict(m_res_net_acc), index=[0])
m_data4['model_name'] = 'ResNetAcc'
m_data = pd.concat([m_data1, m_data2, m_data3, m_data4])
m_data = m_data[['model_name', 'human', 'model', 'comb none', 'comb confusion',
       'comb temperature scaling', 'comb ensemble temperature scaling',
       'comb imax binning']]

m_data

,model_name,human,model,comb none,comb confusion,comb temperature scaling,comb ensemble temperature scaling,comb imax binning
0,DenseNet,4.466667,0.966667,1.666667,1.300000,1.333333,1.366667,1.400000
0,PreresNet,4.466667,0.866667,1.600000,1.066667,1.266667,1.466667,1.333333
0,RLowAcc,4.466667,0.933333,1.533333,1.100000,1.466667,1.466667,1.400000
0,ResNetAcc,4.466667,0.933333,1.633333,1.100000,1.266667,1.433333,1.333333


In [ ]:
# m_dense_net_cal = pd.read_csv('output-team/densenet-bc-L190-k40_calibration.csv')
# m_preres_net_cal = pd.read_csv('output-team/preresnet-110_calibration.csv')
# m_r_low_acc_cal = pd.read_csv('output-team/r_low_acc_calibration.csv')
# m_resnet_cal = pd.read_csv('output-team/resnet-110_calibration.csv')

# m_dense_net_cal['metric'] = m_dense_net_cal['metric'].replace(['ECE (M)', 'ECE (W)'],'ECE')
# m_dense_net_cal['metric'] = m_dense_net_cal['metric'].replace(['cwECE (WT)', 'cwECE (MT)', 'cwECE (WNT)', 'cwECE (MNT)'],'cwECE')

# m_preres_net_cal['metric'] = m_preres_net_cal['metric'].replace(['ECE (M)', 'ECE (W)'],'ECE')
# m_preres_net_cal['metric'] = m_preres_net_cal['metric'].replace(['cwECE (WT)', 'cwECE (MT)', 'cwECE (WNT)', 'cwECE (MNT)'],'cwECE')

# m_r_low_acc_cal['metric'] = m_r_low_acc_cal['metric'].replace(['ECE (M)', 'ECE (W)'],'ECE')
# m_r_low_acc_cal['metric'] = m_r_low_acc_cal['metric'].replace(['cwECE (WT)', 'cwECE (MT)', 'cwECE (WNT)', 'cwECE (MNT)'],'cwECE')

# m_resnet_cal['metric'] = m_resnet_cal['metric'].replace(['ECE (M)', 'ECE (W)'],'ECE')
# m_resnet_cal['metric'] = m_resnet_cal['metric'].replace(['cwECE (WT)', 'cwECE (MT)', 'cwECE (WNT)', 'cwECE (MNT)'],'cwECE')

# m_dense_net_data = m_dense_net_cal.groupby('metric').mean()
# m_preres_net_data = m_preres_net_cal.groupby('metric').mean()
# m_r_low_acc_data = m_r_low_acc_cal.groupby('metric').mean()
# m_resnet_data = m_resnet_cal.groupby('metric').mean()

# m_dense_net_data['name'] = 'DenseNet'
# m_preres_net_data['name'] = 'PreResNet'
# m_r_low_acc_data['name'] = 'RLowAcc'
# m_resnet_data['name'] = 'ResNet'

# pd.concat([m_dense_net_data, m_preres_net_data, m_r_low_acc_data, m_resnet_data]).groupby([ 'metric', 'name']).mean()

**OBSERVATIONS**
1. The human model accuracy given by Neural Network for Calibrated Probabilities has a very minor changes. Which is 0.2% better than our model given in the paper.

2. Accuracy across other models except human shows more than 1% change in the model. 

3. The model accuracy has been inceresed by 2% in DenseNet, PreresNet and ResNet. But in the case of RLow this difference is around 10%.

4. Human + machine combination show the same 1% increase accuracy.

5. Calibration of human-machine (P+L) combinations with confusion matrix show even better performance than previous model and human + machine model.

6. comb temperature scaling, comb ensemble temperature scaling and comb imax binning can clearly show around 1% better performance.
